In [1]:
# Packages
# generic imports
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import time
import geemap
import geojson
import ee
ee.Initialize()

# specific imports
from typing import Any, Dict, List, Optional
from geojson import Polygon, Feature, FeatureCollection, dump
from shapely.geometry import Polygon
from dateutil.relativedelta import *
from google.cloud import storage
from logging import Logger, getLogger
from googleapiclient.discovery import build
from re import sub
from ctypes import ArgumentError
from functools import partial
from dateutil.parser import parse

# custom functionality import without requirement to pip install package
local_path = r"c:\Users\white_rn\Documents\GitHub\ee-packages-py"  # path to local GitHub clone
sys.path.append(local_path)
from eepackages.applications.bathymetry import Bathymetry
from eepackages import tiler

logger: Logger = getLogger(__name__)

C:\Users\white_rn\AppData\Local\Temp\ipykernel_7328\1492826932.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Settings
print('Settings')
# Folder and file paths
main_fol = r"p:\11209821-cmems-global-sdb" # name of the main local folder 
aoi_fol = os.path.join(main_fol, "00_miscellaneous", "AOIs") # name of the folder containing the AoI
aoi_polygon_fol = os.path.join(main_fol, "00_miscellaneous", "AOI_polygons") # name of the folder containing the AoI polygons
output_fol = os.path.join(main_fol, "01_intertidal", "02_data") # name of the folder containing the output

bucket = "cmems-sdb" # name of the Google Cloud Storage bucket to store files in the cloud

# Area of Interest (AoI) settings
project_name = "AOI_GER_WaddenSea" # name of the project AoI

# Composite image settings
mode = "intertidal_improved" # specify mode, either "intertidal" or "subtidal"
start_date = "2021-01-01" # start date of the composites
stop_date = "2021-02-01" # end date of the composites
compo_int = 12 # composite interval [months]
compo_len = 12 # composite length [months]
scale = 100  # output resolution of the image [m]
crs = "EPSG:4326" # output projection of the image

# Tiling settings
zoomed_list = [9, 10, 11] # list with zoom levels to be inspected
sel_zoom = 2 # idx of chosen tile level in zoomed_list (inspect the map to chose it accordingly), z9 too big for in memory computations

# Load google credentials
credential_file = os.path.join(main_fol, "00_miscellaneous", "KEYS", "bathymetry-543b622ddce7.json") # Cloud Storage credential key
if not credential_file == "":  
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(credential_file)

# Get the Area of Interest
print('Area of Interest')
with open(os.path.join(aoi_fol, project_name + ".geojson")) as f:
    AoIjson = geojson.loads(f.read())

# Get the bounds of the AoI
print('Bounds')
bounds = ee.Geometry(AoIjson["features"][0]["geometry"])

# Get the tiles for the AoI
print('Tiles')
tiles_list = [tiler.get_tiles_for_geometry(geometry=bounds, zoom=ee.Number(zoom)) for zoom in zoomed_list]
tiles = tiles_list[sel_zoom]
tile = ee.Feature(tiles.filterMetadata("tx", "equals", "1065.0").filterMetadata("ty", "equals", "660.0").first())

# Get start and stop dates 
print('Dates')
start=ee.String(start_date)
stop=ee.String(stop_date)


Settings
Area of Interest
Bounds
Tiles
Dates


In [3]:
# Plot AoI and tiles
'''
# Plot AoI and tiles
print('Create map')
Map = geemap.Map()

print('Center map')
Map.centerObject(bounds, 9)

print('Add Area of Interest')
Map.addLayer(bounds, {}, "AoI")

print('Add all tiles')
for tiles_, zoom in zip(tiles_list, zoomed_list):
	Map.addLayer(tiles_, {}, "Tiles {}".format(zoom))
	
print('Add selected tiles')
Map.addLayer(tiles, {}, "Selected Tiles")

print('Add selected tile')
Map.addLayer(tile, {}, "Selected Tile")

print('Show map')
Map
'''


'\n# Plot AoI and tiles\nprint(\'Create map\')\nMap = geemap.Map()\n\nprint(\'Center map\')\nMap.centerObject(bounds, 9)\n\nprint(\'Add Area of Interest\')\nMap.addLayer(bounds, {}, "AoI")\n\nprint(\'Add all tiles\')\nfor tiles_, zoom in zip(tiles_list, zoomed_list):\n\tMap.addLayer(tiles_, {}, "Tiles {}".format(zoom))\n\t\nprint(\'Add selected tiles\')\nMap.addLayer(tiles, {}, "Selected Tiles")\n\nprint(\'Add selected tile\')\nMap.addLayer(tile, {}, "Selected Tile")\n\nprint(\'Show map\')\nMap\n'

In [4]:
# Functions to compute the NDWI and Weighted NDWI
def NDWI(image):
	""" Calculates the NDWI of an image
	
	:param image: an image
	:type: image: ee.Image
	:return: An image with a single ndwi band
	:rtype: ee.Image
	"""
	
	ndwi = image.normalizedDifference(['green', 'nir']).rename("ndwi")

	return image.addBands(ndwi)

def ndwiWater(image):
	""" Identifies water in an image based on the ndwi and a threshold value
	
	:param image: an image
	:type: image: ee.Image
	:return: An image where water is identified as 1 and non-water as 0
	:rtype: ee.Image
	"""
	
	# find the water using a fixed NDWI threshold
	NDWIWater = image.select("ndwi").gt(ndwiThreshold).rename("ndwi_water")

	#TODO: check influence of this waterclean function on output
	# clean up the image 
	# look into dilation in morphology module
	connectedPixelSize = 512 #this can cause an internal error if set too high (1024 created errors for some cells)
	def waterClean(image, size):
		connectedPixels = image.int().connectedPixelCount(size)
		pixelCountWhere = image.where(connectedPixels.lt(size), 0)
		return pixelCountWhere

	#imageCleaned = NDWIWater
	imageCleaned = waterClean(NDWIWater, connectedPixelSize)
	imageCleanedWeight = imageCleaned.multiply(image.select("weight")).rename("ndwi_waterweight")

	time = image.get('system:time_start')

	imageCleaned.set('system:time_start', time)
	imageCleanedWeight.set('system:time_start', time)
	
	return image.addBands(imageCleaned).addBands(imageCleanedWeight)


In [5]:
# %%
# Function to compute the intertidal depth for tile
bounds: ee.Geometry = ee.Feature(tile).geometry().bounds(1)
sdb: Bathymetry = Bathymetry()
zoom: ee.String = ee.String(tile.get("zoom"))
tx: ee.String = ee.String(tile.get("tx"))
ty: ee.String = ee.String(tile.get("ty"))
tile_name: ee.String = ee.String("z").cat(zoom).cat("_x").cat(tx).cat("_y").cat(ty).replace("\.\d+", "", "g")
img_fullname: ee.String = ee.String(tile_name).cat("_t").cat(ee.Date(start).millis().format())

image: ee.Image = sdb.compute_intertidal_depth(
    bounds=bounds,
    start=start,
    stop=stop,
    scale=tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom"))).multiply(5), # scale to search for clean images
    # missions=['S2', 'L8'],
    # filter: ee.Filter.dayOfYear(7*30, 9*30), # summer-only
    filter_masked=False, 
    # filterMaskedFraction = 0.5,
    # skip_scene_boundary_fix=False,
    # skip_neighborhood_search=False,
    neighborhood_search_parameters={"erosion": 0, "dilation": 0, "weight": 50},
    bounds_buffer=0,
    water_index_min=-0.05,
    water_index_max=0.15,
    # lowerCdfBoundary=45,
    # upperCdfBoundary=50,
    # cloud_frequency_threshold_data=0.15, 
    clip = True
)# .reproject(ee.Projection("EPSG:3857").atScale(90))

image = image.set(
    "fullname", img_fullname,
    "system:time_start", ee.Date(start).millis(),
    "system:time_stop", ee.Date(stop).millis(),
    "zoom", zoom,
    "tx", tx,
    "ty", ty
)

# TODO: understand the CDFquality score weighting 

In [6]:
# %%
# Compute the NDWI and Weighted NDWI for tile
# TODO: # implement in ee_packages_py code and check..

# compute NDWI for imagecollection
# ndwi = sdb._refined_images.first().normalizedDifference(['green', 'nir']).rename("ndwi") # trial single image
# water_index_min=-0.05
# water_index_max=0.15
# ndwi = ndwi.clamp(water_index_min, water_index_max)

# map NDWI over entire imagecollection 
NDWICollection = sdb._refined_images.map(NDWI)

# water threshold
# TODO: replace by otsu (see ppt on intertidal algs decision otsu from Kel Markert)
ndwiThreshold = 0.2

# map to absolute pixel values (1 for water) and (0 for non-water) in NDWI images
NDWICollectionMapped = NDWICollection.map(ndwiWater)

In [7]:
# %%
# calculate water occurrence of the collection
waterReduceSum = NDWICollectionMapped.select("ndwi_water").reduce(ee.Reducer.sum()).int16().rename('waterOccurrenceCount') # count number of water occurrences at each pixel
waterReduceSum = waterReduceSum.where(waterReduceSum.eq(1), 0) #remove pixels that only have 1 for water occurence, TODO: QA check - review this
waterPercentage = waterReduceSum.divide(NDWICollectionMapped.size()).multiply(100).rename('waterOccurrencePercentage').addBands(waterReduceSum) #calculates the water occurence % (mean*100)
mask = waterPercentage.select('waterOccurrenceCount').mask() #adds the image collection size as a band
bandCollectionLength = ee.Image.constant(NDWICollectionMapped.size()).uint16().rename('numberOfImagesAnalysed').updateMask(mask)
gridCellWaterOccurrenceOutput = waterPercentage.addBands(bandCollectionLength) #add a band within the water occurrence image that holds the number of images used to calculate the water occurrence

# weighted average (iso mean of water occurence)
# .map(lambda i: i.select("ndwi_water").multiply(i.select("weight")))
# TODO: check ndwi_waterweight output vs .map lambda output
waterReduceSumWeighted = NDWICollectionMapped.select("ndwi_waterweight").reduce(ee.Reducer.sum()).int16().rename('waterOccurrenceCountWeighted') # count number of water occurrences at each pixel
bandCollectionLengthWeighted = NDWICollectionMapped.select("weight").reduce(ee.Reducer.sum()).int16().rename('numberOfImagesAnalysedWeighted') # count number of water occurrences at each pixel
waterPercentageWeighted = NDWICollectionMapped.select("ndwi_waterweight")\
                        .sum()\
                        .divide(NDWICollectionMapped.select("weight").sum())\
                        .multiply(100)\
                        .rename("waterOccurrencePercentageWeighted")
gridCellWaterOccurrenceOutput = gridCellWaterOccurrenceOutput.addBands(waterPercentageWeighted).addBands(waterReduceSumWeighted).addBands(bandCollectionLengthWeighted) #add bands to the water occurrence image

# create median NDWI band 
ndwiMedian = NDWICollectionMapped.select("ndwi_water").median().rename('ndwiMedian') #calculate the median NDWI
gridCellWaterOccurrenceOutput = gridCellWaterOccurrenceOutput.addBands(ndwiMedian) #add ndwiMedian as a band to the water occurrence image

In [8]:
# Create mask based on gridCellWaterOccurrenceOutput (waterOccurrencePercentageWeighted > limit[0] and < limit[1])
limit = [0.05, 99.95]
mask = gridCellWaterOccurrenceOutput.select('waterOccurrencePercentageWeighted').gt(limit[0]).And(gridCellWaterOccurrenceOutput.select('waterOccurrencePercentageWeighted').lt(limit[1]))

# Mask the image with the mask
gridCellWaterOccurrenceOutput_masked = gridCellWaterOccurrenceOutput.updateMask(mask)

# Clip the image to the bounds
gridCellWaterOccurrenceOutput_clipped = gridCellWaterOccurrenceOutput_masked.clip(bounds)

# Reproject and rescale the image
gridCellWaterOccurrenceOutput_reprojected = gridCellWaterOccurrenceOutput_clipped.reproject(crs=crs, scale=scale)

In [9]:
tile_metadata = tile.getInfo()['properties']
tx = tile_metadata["tx"]
ty = tile_metadata["ty"]
zoom = tile_metadata["zoom"]

test_str = f"{mode}/z{int(zoom)}/x{int(tx)}/y{int(ty)}/{start_date}-{stop_date}.nc"
print(test_str)


ValueError: invalid literal for int() with base 10: '1065.0'

In [21]:
def tile_to_cloud_storage(
    image: ee.Image,
    tile: ee.Feature,
    scale: int,
    crs: str,
    bucket: str,
    bucket_path: str,
    overwrite: bool
) -> Optional[ee.batch.Task]:
    with build('storage', 'v1') as storage:
        res = storage.objects().list(bucket=bucket, prefix="/".join(bucket_path.split("/")[:-1])).execute()
    if not overwrite:
        try:
            object_exists = any(map(lambda item: item.get("name").startswith(bucket_path), res.get("items")))
        except AttributeError:
            object_exists = False
        if object_exists:
            logger.info(f"object {bucket_path} already exists in bucket {bucket}, skipping")
            return
        
    task: ee.batch.Task = ee.batch.Export.image.toCloudStorage(
        image,
        bucket=bucket,
        description=bucket_path.replace("/", "_"),
        fileNamePrefix=bucket_path,
        region=tile.geometry(),
        scale=scale,
        crs=crs,
        fileFormat='GeoTIFF',
        formatOptions= {'cloudOptimized': True}, # enables easy QGIS plotting
        maxPixels= 1e10
    )
    task.start()
    return task

# Get tile metadata
tile_metadata = tile.getInfo()['properties']
tx = float(tile_metadata["tx"])
ty = float(tile_metadata["ty"])
zoom = float(tile_metadata["zoom"])

# Export the image to the cloud
tile_to_cloud_storage(
    image=gridCellWaterOccurrenceOutput_reprojected.select("waterOccurrencePercentageWeighted"),
    tile=tile,
    scale=scale,
    crs=crs,
    bucket=bucket,
    bucket_path='{}/z{:0.0f}/x{:0.0f}/y{:0.0f}/{}_{}'.format(mode, zoom, tx, ty, start_date, stop_date),
    overwrite=True
)

# NOTE: NetCDF export is not supported by Earth Engine, so we export to GeoTIFF instead (which can be converted to NetCDF using GDAL translate)

<Task 2PV64SBFJIV2V3EFPBAIQHAP EXPORT_IMAGE: intertidal_improved_z11_x1065_y660_2021-01-01_2021-02-01 (UNSUBMITTED)>

In [26]:
# Convert GeoTIFF to NetCDF
from osgeo import gdal
import rioxarray as rxr

# File paths
file_path_tif = r'p:\11209821-cmems-global-sdb\01_intertidal\02_data\00_temp\2021-01-01_2021-02-01.tif'
file_path_nc = r'p:\11209821-cmems-global-sdb\01_intertidal\02_data\00_temp\2021-01-01_2021-02-01.nc'

# Convert GeoTIFF to NetCDF
gdal.Translate(file_path_nc, file_path_tif, format='NetCDF')

# Open the GeoTIFF and NetCDF files
data_tif = rxr.open_rasterio(file_path_tif)
data_nc = rxr.open_rasterio(file_path_nc)

In [ ]:
'''
# Clip the image collection to the bounds
NDWIWaterCollection_clipped = NDWICollectionMapped.map(lambda image: image.clip(bounds))

# Reproject and rescale the image collection
NDWICollectionMapped_reprojected = NDWIWaterCollection_clipped.map(lambda image: image.reproject(crs=crs, scale=scale))

# Get the first image
image_clipped = NDWIWaterCollection_clipped.first().select("waterOccurrencePercentageWeighted")
image_reprojected = NDWICollectionMapped_reprojected.first().select("waterOccurrencePercentageWeighted")
'''

In [ ]:
# Get the image information
image_clipped_info = gridCellWaterOccurrenceOutput_clipped.getInfo()
image_reprojected_info = gridCellWaterOccurrenceOutput_reprojected.getInfo()

print(image_clipped_info['bands'][3])
print(image_reprojected_info['bands'][3])

In [ ]:
# Get the image array
array = gridCellWaterOccurrenceOutput_reprojected\
        .select('waterOccurrencePercentageWeighted')\
        .sampleRectangle(region=bounds, defaultValue=-1)\
        .get('waterOccurrencePercentageWeighted')\
        .getInfo()

In [ ]:
array = np.where(array == -1, np.nan, array)
print('Min: {}'.format(np.nanmin(array)))
print('Max: {}'.format(np.nanmax(array)))

In [ ]:
# Plot AoI and tiles
print('Create map')
Map = geemap.Map()

print('Center map')
Map.centerObject(tile, zoomed_list[sel_zoom])

print('Add image clipped')
visParams_NDWI = {'min': 0, 'max': 100, 'palette': ['FF0000','00A6FF']}
Map.addLayer(gridCellWaterOccurrenceOutput_clipped.select('waterOccurrencePercentageWeighted'), visParams_NDWI, 'image clipped')

print('Add image reprojected')
Map.addLayer(gridCellWaterOccurrenceOutput_reprojected.select('waterOccurrencePercentageWeighted'), visParams_NDWI, 'image reprojected')

print('Show map')
Map

In [ ]:
# %%
'''
# Get dates of all images
ee_dates = sdb._raw_images.aggregate_array("system:time_start").map(lambda date: ee.Date(date).format("YYYY-MM-dd HH:mm:ss"))
dates = ee_dates.getInfo()
print(dates)
len(dates)
'''

In [ ]:
# %%
# below I retrieve various band data to check the output...

# %%
# note, scale was adjusted as the original scale was too high for the sampleRectangle function (memory limits as GEE was overcrowed)
# get array and check values
#band_arrs_ndwi = ndwi.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
# band_arrs_ndwi = NDWICollection.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
#band_arrs_ndwi = NDWICollectionMapped.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arrs_ndwi = gridCellWaterOccurrenceOutput.reproject(crs = ee.Projection('EPSG:4326'), scale=120).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('waterOccurrenceCount')
np_arr_ndwi2 = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi2.shape)
print(np_arr_ndwi2)

# %%
# plot data array
# TODO: note the blurryness, try to use the weights to get rid of it (clouds)
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi2, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
# get array and check values
#band_arrs_ndwi = ndwi.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
# band_arrs_ndwi = NDWICollection.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arrs_ndwi = NDWICollectionMapped.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
#band_arrs_ndwi = gridCellWaterOccurrenceOutput.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('weight')
np_arr_ndwi2 = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi2.shape)
print(np_arr_ndwi2)

# %%
np.max(np_arr_ndwi2)

# %%
# plot data array
# TODO: note the blurryness, try to use the weights to get rid of it (clouds)
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi2, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
# get array and check values
#band_arrs_ndwi = ndwi.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
# band_arrs_ndwi = NDWICollection.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arrs_ndwi = NDWICollectionMapped.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
#band_arrs_ndwi = gridCellWaterOccurrenceOutput.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('ndwi_waterweight')
np_arr_ndwi2 = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi2.shape)
print(np_arr_ndwi2)

# %%
# plot data array
# TODO: note the blurryness, try to use the weights to get rid of it (clouds)
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi2, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
gridCellWaterOccurrenceOutput.getInfo()

# %%
# get array and check values
#band_arrs_ndwi = ndwi.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
# band_arrs_ndwi = NDWICollection.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
#band_arrs_ndwi = NDWICollectionMapped.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arrs_ndwi = gridCellWaterOccurrenceOutput.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('ndwiMedian')
np_arr_ndwi2 = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi2.shape)
print(np_arr_ndwi2)

# %%
# plot data array
# TODO: note the blurryness, try to use the weights to get rid of it (clouds)
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi2, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
# get array and check values
#band_arrs_ndwi = ndwi.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
# band_arrs_ndwi = NDWICollection.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
#band_arrs_ndwi = NDWICollectionMapped.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arrs_ndwi = gridCellWaterOccurrenceOutput.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('waterOccurrencePercentage')
np_arr_ndwi = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi.shape)
print(np_arr_ndwi)

# %%
# plot data array
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi, vmin=0, vmax=100, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
# note, scale was adjusted as the original scale was too high for the sampleRectangle function (memory limits as GEE was overcrowed)
band_arrs_ndwi = waterPercentageWeighted.reproject(crs = ee.Projection('EPSG:4326'), scale=120).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('waterOccurrencePercentageWeighted')
np_arr_ndwi2 = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi2.shape)
print(np_arr_ndwi2)

# %%
band_arrs_ndwi = waterPercentageWeighted.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('waterOccurrencePercentageWeighted')
np_arr_ndwi = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi.shape)
print(np_arr_ndwi)

# %%
# plot data array
# TODO: note the blurryness, try to use the weights to get rid of it (clouds)
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi2, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
# get array and check values
#band_arrs_ndwi = ndwi.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
# band_arrs_ndwi = NDWICollection.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arrs_ndwi = NDWICollectionMapped.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
#band_arrs_ndwi = gridCellWaterOccurrenceOutput.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs_ndwi.get('ndwi_water')
np_arr_ndwi2 = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi2.shape)
print(np_arr_ndwi2)

# %%
# plot data array
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi2, vmin=0, vmax=1, cmap="Greys_r")#vmin=-0.05, vmax=0.15
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
band_arrs = sdb._images_with_statistics.first().reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs.get('ndwi')
np_arr_ndwi = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi.shape)
print(np_arr_ndwi)

# %%
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi, vmin=-0.05, vmax=0.15, cmap="Greys_r")
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%
band_arrs = image.reproject(crs = ee.Projection('EPSG:4326'), scale=scale).sampleRectangle(region=bounds)
band_arr_ndwi = band_arrs.get('ndwi')
np_arr_ndwi = np.array(band_arr_ndwi.getInfo())
print(np_arr_ndwi.shape)
print(np_arr_ndwi)

# %%
plt.figure(figsize=(16,9))
plt.imshow(np_arr_ndwi, vmin=-0.05, vmax=0.15, cmap="Greys_r")
plt.grid(alpha=0.5)
# plt.clim(0,1)
plt.colorbar()
plt.show()

# %%